Binary Logistic Regression Speed-Up in Python with Numba

In [ ]:
# import the necessary libraries
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import math
import numpy as np
import time

# running the sigmoid function
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# the classifier
def if_above(x):
  for i in range(len(x)):
    if x[i] >= 0.5:
      x[i] = 1
    else:
      x[i] = 0
  return x

# applying logistic regression
def calculating_logistic_regression(X_train, weights, bias):
  z = np.dot(X_train, weights) + bias
  y_pred = sigmoid(z)
  return if_above(y_pred)

# comparing ground truth y_true to your predictions y_pred
def compute_loss(y_train, y_pred):
  epsilon = 1e-9
  m = len(y_train)
  loss = 0.0
  y_train = y_train.reshape(-1) # fixing up data
  y_pred = y_pred.reshape(-1)

  for i in prange(m):
      loss += y_train[i] * math.log(y_pred[i] + epsilon) + (1 - y_train[i]) * math.log(1 - y_pred[i] + epsilon)
  # adding epsilon so not exactly 0 and 1

  return -loss / m   # mean of loss

# calculate gradient to use to update the model parameters
def gradient(y_train, y_pred, X_train):
  m = len(y_train)
  difference = y_pred -  y_train.reshape(-1)
  weight_update = np.dot(X_train.T, difference) / m
  bias_update = np.sum(difference) / m
  return weight_update, bias_update

# updating weights and bias with values
def update_parameters(weights, bias, weight_update, bias_update, learning_rate = 0.1):
  for i in range(weights.shape[0]):
    weights[i] -= learning_rate * weight_update[i]
  bias -= learning_rate * bias_update
  return weights, bias

# calculating weights and sigmoid calculation
def fit(X_train, y_train, epochs=200, learning_rate = 0.1):
  # given values
  x_rows, x_columns = X_train.shape
  weights = np.random.rand(x_columns) * 0.01  # initializing weights to small random values
  bias = 0  # initializing bias to zero
  y_train = y_train.reshape(-1, 1)  # Ensuring that y_train is of shape (m, 1)

  epoch_times = [] # Time for each epochs run

  # for a certain amount of epochs apply gradient
  for epoch in range(epochs):
    start_time = time.time()

    # applying logistic regression
    y_pred = calculating_logistic_regression(X_train, weights, bias)

    # updating weights and bias
    weight_update, bias_update = gradient(y_train, y_pred, X_train)
    weights, bias = update_parameters(weights, bias, weight_update, bias_update, learning_rate)

    # calculating loss
    loss = compute_loss(y_train, y_pred)

    end_time = time.time()
    epoch_time = end_time - start_time
    epoch_times.append(epoch_time)

    print("Epoch: {} Loss: {} Time: {:.4f} seconds".format(epoch, loss, epoch_time))

  avg_epoch_time = np.mean(epoch_times[1:]) # except first cause getting set up
  print("Average time per epoch: {:.4f} seconds".format(avg_epoch_time))

  return weights, bias

# predicting with X_test
def predict(X_test, weights, bias):
  return calculating_logistic_regression(X_test, weights, bias)

# load the breast cancer dataset
X, y = load_breast_cancer(return_X_y=True)

# split the train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=23)

# now my part comes in will replace the previous LogisticRegression with my own
weights, bias = fit(X_train, y_train)

# Prediction
y_pred = predict(X_test, weights, bias)

# Compaaring with previous y_test
acc = accuracy_score(y_test, y_pred)
print("Logistic Regression model accuracy (in %):", acc * 100)

Epoch: 0 Loss: 7.8793955813400425 Time: 0.0130 seconds
Epoch: 1 Loss: 12.843870254606308 Time: 0.0082 seconds
Epoch: 2 Loss: 12.843870254606308 Time: 0.0048 seconds
Epoch: 3 Loss: 7.8793955813400425 Time: 0.0091 seconds
Epoch: 4 Loss: 12.843870254606308 Time: 0.0054 seconds
Epoch: 5 Loss: 7.8793955813400425 Time: 0.0053 seconds
Epoch: 6 Loss: 12.843870254606308 Time: 0.0053 seconds
Epoch: 7 Loss: 12.843870254606308 Time: 0.0053 seconds
Epoch: 8 Loss: 7.8793955813400425 Time: 0.0052 seconds
Epoch: 9 Loss: 12.843870254606308 Time: 0.0053 seconds
Epoch: 10 Loss: 7.8793955813400425 Time: 0.0057 seconds
Epoch: 11 Loss: 12.843870254606308 Time: 0.0053 seconds
Epoch: 12 Loss: 11.24977288245658 Time: 0.0052 seconds
Epoch: 13 Loss: 7.8793955813400425 Time: 0.0052 seconds
Epoch: 14 Loss: 12.843870254606308 Time: 0.0052 seconds
Epoch: 15 Loss: 7.8793955813400425 Time: 0.0051 seconds
Epoch: 16 Loss: 12.843870254606308 Time: 0.0052 seconds
Epoch: 17 Loss: 2.2317363200096243 Time: 0.0052 seconds
Epo

<ipython-input-124-322b08496249>:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch: 31 Loss: 2.2772819592139024 Time: 0.0076 seconds
Epoch: 32 Loss: 5.101111589879124 Time: 0.0179 seconds
Epoch: 33 Loss: 7.287302271684437 Time: 0.0089 seconds
Epoch: 34 Loss: 12.843870254606308 Time: 0.0075 seconds
Epoch: 35 Loss: 2.004008123988235 Time: 0.0074 seconds
Epoch: 36 Loss: 9.883403706328238 Time: 0.0074 seconds
Epoch: 37 Loss: 7.8793955813400425 Time: 0.0075 seconds
Epoch: 38 Loss: 12.843870254606308 Time: 0.0075 seconds
Epoch: 39 Loss: 7.196210993275882 Time: 0.0076 seconds
Epoch: 40 Loss: 12.843870254606308 Time: 0.0075 seconds
Epoch: 41 Loss: 2.1406450416010685 Time: 0.0076 seconds
Epoch: 42 Loss: 12.707233336993474 Time: 0.0112 seconds
Epoch: 43 Loss: 7.8793955813400425 Time: 0.0106 seconds
Epoch: 44 Loss: 12.843870254606308 Time: 0.0072 seconds
Epoch: 45 Loss: 7.332847910888714 Time: 0.0075 seconds
Epoch: 46 Loss: 12.843870254606308 Time: 0.0075 seconds
Epoch: 47 Loss: 2.8238296296652363 Time: 0.0079 seconds
Epoch: 48 Loss: 12.843870254606308 Time: 0.0155 second

In [ ]:
# parallelized version
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import math
from numba import njit, prange
import numpy as np
import time

@njit
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

@njit(parallel=True)
def if_above(x):
  for i in prange(len(x)):
    if x[i] >= 0.5:
      x[i] = 1
    else:
      x[i] = 0
  return x

# applying logistic regression
@njit
def calculating_logistic_regression(X_train, weights, bias):
  z = np.dot(X_train, weights) + bias
  y_pred = sigmoid(z)
  return if_above(y_pred)

# comparing ground truth y_true to your predictions y_pred
@njit(parallel=True, fastmath = True)
def compute_loss(y_train, y_pred):
  epsilon = 1e-9
  m = len(y_train)
  loss = 0.0
  y_train = y_train.reshape(-1) # fixing up data
  y_pred = y_pred.reshape(-1)

  for i in prange(m):
      loss += y_train[i] * math.log(y_pred[i] + epsilon) + (1 - y_train[i]) * math.log(1 - y_pred[i] + epsilon)
  # adding epsilon so not exactly 0 and 1

  return -loss / m   # mean of loss

# calculate gradient to use to update the model parameters
@njit
def gradient(y_train, y_pred, X_train):
  m = len(y_train)
  difference = y_pred -  y_train.reshape(-1)
  weight_update = np.dot(X_train.T, difference) / m
  bias_update = np.sum(difference) / m
  return weight_update, bias_update

@njit(parallel=True, fastmath = True)
def update_parameters(weights, bias, weight_update, bias_update, learning_rate = 0.1):
  for i in prange(weights.shape[0]):
    weights[i] -= learning_rate * weight_update[i]
  bias -= learning_rate * bias_update
  return weights, bias

# calculating weights and sigmoid calculation
def fit(X_train, y_train, epochs=200, learning_rate = 0.1):
  # given values
  x_rows, x_columns = X_train.shape # grabbing the columns this will be # of weights
  weights = np.random.rand(x_columns) * 0.01  # initializing weights to small random values
  bias = 0  # initializing bias to zero
  y_train = y_train.reshape(-1, 1)  # Ensuring that y_train is of shape (m, 1)

  epoch_times = [] # Time for each epochs run

  # for a certain amount of epochs apply gradient
  for epoch in range(epochs):
    start_time = time.time() # let's start the epochs going

    # applying logistic regression getting a y_pred
    y_pred = calculating_logistic_regression(X_train, weights, bias)

    # after we will apply the gradient to find out how to update weights and bias
    weight_update, bias_update = gradient(y_train, y_pred, X_train)
    weights, bias = update_parameters(weights, bias, weight_update, bias_update, learning_rate)

    # calculating loss
    loss = compute_loss(y_train, y_pred)

    # time ended
    end_time = time.time()
    epoch_time = end_time - start_time
    epoch_times.append(epoch_time)

    print("Epoch: {} Loss: {} Time: {:.4f} seconds".format(epoch, loss, epoch_time))

  avg_epoch_time = np.mean(epoch_times[1:]) # except first cause getting set up
  print("Average time per epoch: {:.4f} seconds".format(avg_epoch_time))

  return weights, bias

@njit
def predict(X_test, weights, bias):
  return calculating_logistic_regression(X_test, weights, bias)

# load the breast cancer dataset
X, y = load_breast_cancer(return_X_y=True)

# split the train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=23)

# now my part comes in will replace the previous LogisticRegression with my own
weights, bias = fit(X_train, y_train)

# Prediction
y_pred = predict(X_test, weights, bias)

acc = accuracy_score(y_test, y_pred)
print("Logistic Regression model accuracy (in %):", acc * 100)


Epoch: 0 Loss: 7.87939554988054 Time: 3.8976 seconds
Epoch: 1 Loss: 12.843870254606342 Time: 0.8492 seconds
Epoch: 2 Loss: 12.843870254606351 Time: 0.0002 seconds
Epoch: 3 Loss: 7.879395549880542 Time: 0.0007 seconds
Epoch: 4 Loss: 12.843870254606351 Time: 0.0007 seconds
Epoch: 5 Loss: 7.879395549880542 Time: 0.0006 seconds
Epoch: 6 Loss: 12.843870254606351 Time: 0.0006 seconds
Epoch: 7 Loss: 12.843870254606351 Time: 0.0006 seconds
Epoch: 8 Loss: 7.879395549880542 Time: 0.0006 seconds
Epoch: 9 Loss: 12.843870254606351 Time: 0.0006 seconds
Epoch: 10 Loss: 7.879395549880542 Time: 0.0006 seconds
Epoch: 11 Loss: 12.843870254606351 Time: 0.0006 seconds
Epoch: 12 Loss: 11.24977288227478 Time: 0.0006 seconds
Epoch: 13 Loss: 7.879395549880542 Time: 0.0008 seconds
Epoch: 14 Loss: 12.843870254606351 Time: 0.0006 seconds
Epoch: 15 Loss: 7.879395549880542 Time: 0.0006 seconds
Epoch: 16 Loss: 12.843870254606351 Time: 0.0006 seconds
Epoch: 17 Loss: 2.231736311462807 Time: 0.0006 seconds
Epoch: 18 Lo

Explanation
- Process: I first fit my x_train data by running logistical regression on it and adjusting the weights and bias for each epochs accordingly (gradient) to better adjust the y_pred with the y_train. I then applied the same logistical regression steps with the weights and bias discovered from before to the x_test and compared the y_pred_test to the y_test results.
(This was sooo cool to open up and ACTUALLY understand AI functions that I had seen before).
- For my parallelization, my non-parallelized version had an average time per epoch of 0.0066 seconds, and my parallelized version had an average time per epoch of 0.0048 seconds.
- Interestingly enough, I at first saw an increase in time (increase of ~0.02 seconds) every time I added a new @njit(parallel=True) to replace a numpy function where there was not heavy complex operations. The overhead of using numba was seen. I think the reason for this is that Numba is lot more beneficial when you are doing really complex heavy operations in for loops instead of replacing Numpy's vectorized functions already. I saw a huge time decrease when I switched out my old code for computing loss to my new code (went from 0.0215 to 0.0048 seconds)

```
# old code
epsilon = 1e-9
  loss = y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon) # added epsilon so not exactly 0 and 1
  return -np.mean(loss) can we make this into a for loop
```

```
# new code
@njit(parallel=True, fastmath = True)
def compute_loss(y_train, y_pred):
  epsilon = 1e-9
  m = len(y_train)
  loss = 0.0
  y_train = y_train.reshape(-1) # fixing up data
  y_pred = y_pred.reshape(-1)
    
  for i in prange(m):
      loss += y_train[i] * math.log(y_pred[i] + epsilon) + (1 - y_train[i]) * math.log(1 - y_pred[i] + epsilon)

  return -loss / m
```

- Using Numba arguments, I played around with different parameters and saw an increase when I used "fastmath = True". This was specifically great for the floating-point optimizations seen in compute_loss.
- Also, in my Numba code, I use prange with the parallel=True. This creates threads to process the for loop functions. But, since a lot of the functions for my logistic functions did not involve a for loop there were not a lot of possibilities/ places for Numba functions.  
- One last thing I noticed was an overflow that would occur "RuntimeWarning: overflow encountered in exp return 1 / (1 + np.exp(-x))." I learned, after I looked into the sigmoid function more, that it is a result from x being a very large negative value and e^-x exceeding the max floating-point number. I was considering clipping the values (creating bounds), but I was not sure how low I should set the bar. So I left it as is for now.


Results
- Before parallelization: average time per epoch = 0.0066 seconds, model accuracy = 85.09%
- After parallelization: average time per epoch = 0.0048 seconds, model accuracy = 92.11%